In [2]:
# 전쳐리
# db 

In [3]:
# 전처리 
import pandas as pd 
import numpy as np
from datetime import datetime, timedelta
roadkill_df = pd.read_csv('한국도로공사_로드킬 데이터 정보_20250501.csv', encoding='cp949')
roadkill_df.columns = (
    roadkill_df.columns
    .str.replace('\ufeff', '', regex=True)
    .str.strip()
)
roadkill_df.head()
roadkill_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   본부명     59 non-null     object 
 1   지사명     59 non-null     object 
 2   노선명     59 non-null     object 
 3   구간      59 non-null     object 
 4   방 향     59 non-null     object 
 5   5km     59 non-null     int64  
 6   발생건수    59 non-null     int64  
 7   위도      59 non-null     float64
 8   경도      59 non-null     float64
dtypes: float64(2), int64(2), object(5)
memory usage: 4.3+ KB


In [4]:
print(roadkill_df.columns.tolist())

['본부명', '지사명', '노선명', '구간', '방 향', '5km', '발생건수', '위도', '경도']


In [5]:
# 함수: timestamp, status 

In [6]:
def add_timestamp(df: pd.DataFrame,
                  start_dt: str | None = None,
                  step_seconds: int = 240,
                  as_string: bool = True,
                  **kwargs) -> pd.DataFrame:
    # return_string 별칭 허용
    if 'return_string' in kwargs:
        as_string = kwargs.pop('return_string')

    out = df.copy()
    base = datetime.now().replace(microsecond=0) if start_dt is None else pd.to_datetime(start_dt).to_pydatetime()
    times = [base + timedelta(seconds=i * step_seconds) for i in range(len(out))]
    if as_string:
        out['timestamp'] = pd.to_datetime(times).strftime('%Y-%m-%d %H:%M:%S')
    else:
        out['timestamp'] = pd.to_datetime(times)
    return out

In [7]:
def apply_status_rules_dups_only(
    df: pd.DataFrame,
    group_cols=('위도','경도'),
    time_col='timestamp',
    status_col='status',
    keep_order=True
) -> pd.DataFrame:
    """
    규칙(중복 좌표에만 적용):
      1) 첫 이벤트 = 0
      2) 첫 이벤트 시각으로부터 ≤ 1시간 내 나중 이벤트 = 1
      3) '1'이 찍힌 시각으로부터 ≥ 24시간 후 나중 이벤트 = 2
    단일 좌표(해당 좌표가 1건뿐)는 status 변경 없음.
    """
    out = df.copy()
    orig_idx = out.index
    out[time_col] = pd.to_datetime(out[time_col], errors='coerce')

    # 중복 좌표만 추려서 처리
    sizes = out.groupby(list(group_cols), sort=False)[time_col].transform('size')
    dup_mask = sizes >= 2
    work = out.loc[dup_mask].sort_values(list(group_cols) + [time_col]).copy()
    if work.empty:
        return out  # 중복 좌표가 없다면 원본 그대로

    H1  = pd.Timedelta(hours=1).value      # ns
    H24 = pd.Timedelta(hours=24).value     # ns

    def assign(g: pd.DataFrame) -> pd.DataFrame:
        ts = g[time_col].to_numpy('datetime64[ns]')
        st = np.zeros(len(g), dtype=np.int8)  # 기본 0 (첫 이벤트 0)

        if len(g) >= 2:
            # 첫 이벤트 기준 경과 시간(ns)
            d = (ts - ts[0]).astype('timedelta64[ns]').astype('int64')

            # 1시간 이내(>0, ≤1h) → 1
            within_1h = (d > 0) & (d <= H1)
            st[within_1h] = 1

            # 첫 '1' 이후 24시간 지난 시점부터 → 2
            idx1 = np.flatnonzero(within_1h)
            if idx1.size > 0:
                i1 = idx1[0]
                d2 = (ts - ts[i1]).astype('timedelta64[ns]').astype('int64')
                i2 = np.searchsorted(d2, H24, side='left')
                if i2 < len(g):
                    st[i2:] = 2

        g[status_col] = st
        return g

    work = work.groupby(list(group_cols), group_keys=False, sort=False).apply(assign)
    out.loc[work.index, status_col] = work[status_col].astype('int8')

    return out.loc[orig_idx] if keep_order else out

In [8]:
roadkill_info2  = roadkill_df[['지사명', '노선명', '구간', '방 향', '발생건수', '위도', '경도']].copy()
# status 랜덤으로 주기 
np.random.seed(42)
# 0=사망, 1=발견, 2=재발견 (동일 확률)
roadkill_info2['status'] = np.random.choice([0, 1, 2], size=len(roadkill_info2)).astype('int8')

# insert되는 timestamp추가 (일단은 임의의 값)
roadkill_info2 = add_timestamp(roadkill_info2, step_seconds=240, return_string=True)

# 상태 변경하기 
roadkill_info2 = apply_status_rules_dups_only(roadkill_info2)

roadkill_info2.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_14552\2222811637.py:53: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  work = work.groupby(list(group_cols), group_keys=False, sort=False).apply(assign)


,지사명,노선명,구간,방 향,발생건수,위도,경도,status,timestamp
0,영동,경부선,260~265,부산,3,36.317750,127.556418,2,2025-08-26 09:06:08
1,대전,경부선,270~275,부산,3,36.349629,127.466302,0,2025-08-26 09:10:08
2,대전,경부선,285~290,부산,3,36.465756,127.418342,2,2025-08-26 09:14:08
3,대전,경부선,295~300,부산,3,36.550930,127.431874,2,2025-08-26 09:18:08
4,천안,경부선,305~310,서울,3,36.625360,127.383686,0,2025-08-26 09:22:08


In [9]:
roadkill_info2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 0 to 58
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   지사명        59 non-null     object        
 1   노선명        59 non-null     object        
 2   구간         59 non-null     object        
 3   방 향        59 non-null     object        
 4   발생건수       59 non-null     int64         
 5   위도         59 non-null     float64       
 6   경도         59 non-null     float64       
 7   status     59 non-null     int8          
 8   timestamp  59 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(1), int8(1), object(4)
memory usage: 4.2+ KB


In [10]:
# 중복데이터 확인 
# 완전 일치 기준 중복 여부
mask_exact = roadkill_info2.duplicated(subset=['위도','경도'], keep=False)
dups_exact = (roadkill_info2[mask_exact]
              .sort_values(['위도','경도','timestamp']))
print("완전 일치 중복 샘플:")
print(dups_exact.head(10))

완전 일치 중복 샘플:
   지사명          노선명         구간 방 향  발생건수         위도          경도  status  \
9   순천          남해선      5~10   순천     3  34.990320  127.528375       0   
10  순천          남해선      5~10   부산     3  34.990320  127.528375       1   
16  부안         서해안선     65~70   목포     4  35.357142  126.583441       0   
17  부안         서해안선   120~125   목포     3  35.357142  126.583441       1   
45  진천          중부선   260~265   남이     5  36.708802  127.437539       0   
46  진천          중부선   260~265   하남     3  36.708802  127.437539       1   
37  공주  서산영덕선(대전당진)     35~40   대전     3  36.782578  126.689747       0   
41  보은  서산영덕선(청주상주)      5~10   영덕     3  36.782578  126.689747       1   
51  진천          중부선   290~295   남이     3  36.947610  127.471406       0   
52  진천          중부선   290~295   하남     3  36.947610  127.471406       1   

             timestamp  
9  2025-08-26 09:42:08  
10 2025-08-26 09:46:08  
16 2025-08-26 10:10:08  
17 2025-08-26 10:14:08  
45 2025-08-26 12:06:08  
46 2025-08-

In [11]:
# 이걸 쿼리에 올리기 

In [14]:
# roadkill_loc (위도+경도 PK), roadkill_freq (지사명 PK) 생성 및 적재

import pandas as pd
import pymysql
from sqlalchemy import create_engine, text

# 1. MySQL 연결 
db_config = {
    'host': 'localhost',
    'user': 'root',
    'passwd': '1234',
    'db': 'roadkill_db',
    'charset': 'utf8'
}

# sqlalchemy engine 
engine = create_engine(
    f"mysql+pymysql://{db_config['user']}:{db_config['passwd']}@{db_config['host']}/{db_config['db']}?charset={db_config['charset']}"
)


# 3. DB테이블 생성
with engine.begin() as conn:
    conn.execute(text("DROP TABLE IF EXISTS roadkill_loc"))
    conn.execute(text("""
        CREATE TABLE roadkill_loc (
            지사명       VARCHAR(100) NOT NULL,
            위도         DOUBLE NOT NULL,
            경도         DOUBLE NOT NULL,
            status       TINYINT NOT NULL COMMENT '0=발견,1=재발견,2=죽음',
            `timestamp`  DATETIME NOT NULL,
            PRIMARY KEY (위도, 경도, status, `timestamp`),
            INDEX idx_ts (`timestamp`)
        ) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;
    """))
        
# 4) 데이터 적재
roadkill_info2.to_sql("roadkill_info", con=engine, if_exists="append", index=False)

print("roadkill_info 적재 완료")

roadkill_info 적재 완료
